In [90]:

vcf_file_address= "22_2k.vcf"#"22_allps_200.vcf"
vcf_file = open(vcf_file_address,'r');


first_het_variant_s1=True 
blockid_blocks_s1=[]
allele_blocks_s1=[]
varpos_blocks_s1=[]

first_het_variant_s2=True 
blockid_blocks_s2=[]
allele_blocks_s2=[]
varpos_blocks_s2=[]
var_blockids1_blocks_s2=[]   # for each var in s2, shows the s1 blockid

header_lines_list=[]
for line in vcf_file:
    line_strip=line.strip()
    if line_strip.startswith('#'):
        header_lines_list.append(line_strip)
        sample_names=line_strip.split('\t')[9:] # last line of header contains sample name
        #print(sample_names)
    else:

        line_parts=line_strip.split('\t')
        gt_flags, s1, s2, s3, s4, s5 = line_parts[8:]

        varpos=int(line_parts[1]) # variant position
        gt_flags_split=gt_flags.split(":")
        
        if "PS" in gt_flags_split:
            idx_ps= gt_flags.split(":").index("PS")
            idx_gt= gt_flags.split(":").index("GT")


            split_s1=s1.split(":")
            blockid_s1 = split_s1[idx_ps]
            allele_s1=split_s1[idx_gt]
            
            if allele_s1=="0/1":allele_s1="./.";blockid_s1="."
                
            if not blockid_s1 == '.':
                if first_het_variant_s1:           # for the first het variant
                    first_het_variant_s1 = False
                    
                    allele_block_s1 = [int(allele_s1[0])]     # alleles of a block
                    varpos_block_s1 = [varpos]               # var pos of a block
                    blockid_blocks_s1.append(int(blockid_s1))   # add the blockid of the first var  to the list of blockids
                    
                else:                              # for the rest of het variants
                    if int(blockid_s1) in blockid_blocks_s1:
                        allele_block_s1.append(int(allele_s1[0]))
                        varpos_block_s1.append(varpos)
                        
                    else: 
                        # add previous block to the list of all blocks
                        allele_blocks_s1.append(allele_block_s1)
                        varpos_blocks_s1.append(varpos_block_s1)

                        # creat new phase block
                        allele_block_s1 = [int(allele_s1[0])]
                        varpos_block_s1 = [varpos]
                        blockid_blocks_s1.append(int(blockid_s1))

                        
                        
            split_s2=s2.split(":")
            blockid_s2 = split_s2[idx_ps]
            allele_s2=split_s2[idx_gt]
            
            if allele_s2=="0/1":allele_s2="./.";blockid_s2="."
                
            if not blockid_s2 == '.':
                if first_het_variant_s2:           # for the first het variant
                    first_het_variant_s2 = False
                    
                    allele_block_s2 = [int(allele_s2[0])]     # alleles of a block
                    varpos_block_s2 = [varpos]               # var pos of a block
                    blockid_blocks_s2.append(int(blockid_s2))   # add the blockid of the first var  to the list of blockids
                    var_blockids1_block_s2=[blockid_s1]
                    
                else:                              # for the rest of het variants
                    if int(blockid_s2) in blockid_blocks_s2:
                        allele_block_s2.append(int(allele_s2[0]))
                        varpos_block_s2.append(varpos)
                        var_blockids1_block_s2.append(blockid_s1)
                        
                    else: 
                        # add previous block to the list of all blocks
                        allele_blocks_s2.append(allele_block_s2)
                        varpos_blocks_s2.append(varpos_block_s2)
                        var_blockids1_blocks_s2.append(var_blockids1_block_s2)

                        # creat new phase block
                        allele_block_s2 = [int(allele_s2[0])]
                        varpos_block_s2 = [varpos]
                        blockid_blocks_s2.append(int(blockid_s2))
                        var_blockids1_block_s2=[blockid_s1]
                       
                        
                        
# add last block to the list of all blocks
allele_blocks_s1.append(allele_block_s1)
varpos_blocks_s1.append(varpos_block_s1)

allele_blocks_s2.append(allele_block_s2)
varpos_blocks_s2.append(varpos_block_s2)
var_blockids1_blocks_s2.append(var_blockids1_block_s2)
                        
    
    
number_phaseblocks_s2= len(allele_blocks_s2)
    
    

    
intersected_blocks_idins1_s2=[] # for each block in s2, the corresponding s1 block id

for var_blockids1_blocks_s2_i in var_blockids1_blocks_s2:
    var_blockids1_blocks_s2_i_set=set(var_blockids1_blocks_s2_i)
    
    if '.' in var_blockids1_blocks_s2_i_set:
        var_blockids1_blocks_s2_i_set.discard('.')
        
    if len(var_blockids1_blocks_s2_i_set)==1:
        intersected_idins1_s2=int(list(var_blockids1_blocks_s2_i_set)[0])
    if len(var_blockids1_blocks_s2_i_set)==0:
        intersected_idins1_s2=0
    if len(var_blockids1_blocks_s2_i_set)>1:
        # find the most frequent blockid in the phaseblock
        intersected_idins1_s2=int(max(var_blockids1_blocks_s2_i_set, key=var_blockids1_blocks_s2_i.count) ) 
        
    intersected_blocks_idins1_s2.append(intersected_idins1_s2)

        

In [91]:
print(blockid_blocks_s1)
for i in range(len(allele_blocks_s1)):
    print(len(allele_blocks_s1[i]))
    
print(blockid_blocks_s2)
for i in range(len(allele_blocks_s2)):
    print(len(allele_blocks_s2[i]))

[16065536, 16406342, 16657810, 16960113]
22
14
12
189
[16065536, 16142229, 16406342, 16460985, 16657810, 16661781, 16960113, 16991976, 17092688, 17180682, 17273838]
20
2
3
11
2
11
12
45
81
16
5


In [108]:

allele_blocks_s2_new=list(allele_blocks_s2)
for block_i in range(number_phaseblocks_s2): # 
#for block_i, allele_block_s2 in enumerate(allele_blocks_s2):
   
    allele_block_s2=allele_blocks_s2[block_i]
    varpos_block_s2=varpos_blocks_s2[block_i]

    intersected_idins1_s2=intersected_blocks_idins1_s2[block_i]
    if intersected_idins1_s2 !=0:
        block_indx_s1= blockid_blocks_s1.index(intersected_idins1_s2)
    varpos_block_s1=varpos_blocks_s1[block_indx_s1]
    allele_block_s1=allele_blocks_s1[block_indx_s1]
    
    intersect_varpos=set(varpos_block_s1) & set(varpos_block_s2)
    similiar_count=0
    for varpos in intersect_varpos:
        allele_s1= allele_block_s1[varpos_block_s1.index(varpos)]
        allele_s2= allele_block_s2[varpos_block_s2.index(varpos)]
        if allele_s1==allele_s2: similiar_count+=1
    #print(similiar_count/len(intersect_varpos))
    if similiar_count < len(intersect_varpos)/2:
        #print("need", similiar_count)
        allele_block_s2_new= [1-i for i in allele_block_s2]
        allele_blocks_s2_new[block_i]=allele_block_s2_new
        


In [15]:
block_i=0
for block_i in range(len(allele_blocks_s2)):
#for block_i, allele_block_s2 in enumerate(allele_blocks_s2):
    check=0

    allele_block_s2=allele_blocks_s2[block_i]
    varpos_block_s2=varpos_blocks_s2[block_i]
    start_pos_block=blockid_blocks_s2[block_i] # varpos_block_s2[0]
    end_pos_block=varpos_block_s2[-1]
    print("sample2",block_i,start_pos_block ,end_pos_block)

    for blockid_index_s1 in range(len(blockid_blocks_s1)):

        blockid_s1_current=blockid_blocks_s1[blockid_index_s1]
        
        if blockid_index_s1 < len(blockid_blocks_s1)-1:
            blockid_s1_next=blockid_blocks_s1[blockid_index_s1+1]
        elif blockid_index_s1 == len(blockid_blocks_s1)-1:
            blockid_s1_next= varpos_blocks_s1[-1][-1]
            

        # find the location of frist_block_s2 in s1_blocks
        if start_pos_block >= blockid_s1_current and  start_pos_block < blockid_s1_next :
            check=blockid_s1_current 

    print("s1",check,'\n')
    

sample2 0 16065536 16090396
s1 16065536 

sample2 1 16142229 16156069
s1 16065536 

sample2 2 16406342 16413241
s1 16406342 

sample2 3 16460985 16476596
s1 16406342 

sample2 4 16657810 16659787
s1 16657810 

sample2 5 16661781 16679716
s1 16657810 

sample2 6 16960113 16963253
s1 16960113 

sample2 7 16991976 17014571
s1 16960113 

sample2 8 17092688 17165240
s1 16960113 

sample2 9 17180682 17231422
s1 16960113 

sample2 10 17273838 17277238
s1 16960113 



0

In [9]:
for block_i in range(len(allele_blocks_s1)):
#for block_i, allele_block_s2 in enumerate(allele_blocks_s2):


    allele_block_s1=allele_blocks_s1[block_i]
    varpos_block_s1=varpos_blocks_s1[block_i]
    start_pos_block=blockid_blocks_s1[block_i] # varpos_block_s2[0]
    end_pos_block=varpos_block_s1[-1]
    print("sample 1",block_i,start_pos_block ,end_pos_block)

sample 1 0 16065536 16156069
sample 1 1 16406342 16476596
sample 1 2 16657810 16678960
sample 1 3 16960113 17277238
